In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [82]:
df = pd.read_csv('teams.csv')
df.dropna()
df

,team,country,year,events,athletes,age,height,weight,medals,prev_medals,prev_3_medals
0,AFG,Afghanistan,1964,8,8,22.0,161.0,64.2,0,0.0,0.0
1,AFG,Afghanistan,1968,5,5,23.2,170.2,70.0,0,0.0,0.0
2,AFG,Afghanistan,1972,8,8,29.0,168.3,63.8,0,0.0,0.0
3,AFG,Afghanistan,1980,11,11,23.6,168.4,63.2,0,0.0,0.0
4,AFG,Afghanistan,2004,5,5,18.6,170.8,64.8,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2139,ZIM,Zimbabwe,2000,19,26,25.0,179.0,71.1,0,0.0,0.0
2140,ZIM,Zimbabwe,2004,11,14,25.1,177.8,70.5,3,0.0,0.0
2141,ZIM,Zimbabwe,2008,15,16,26.1,171.9,63.7,4,3.0,1.0
2142,ZIM,Zimbabwe,2012,8,9,27.3,174.4,65.2,0,4.0,2.3


In [83]:
train,test = train_test_split(df, test_size=0.2,random_state=1)
#random state of 1 says that everytime we run this we get the same split

In [84]:
predictors = ['athletes','events'] 
target = ['medals']

In [85]:
x = train[predictors].copy()
y = train[target].copy()
print(x)
print(y)

      athletes  events
1794         2       2
104        224     105
693          2       2
1260        35      34
1489         5       5
...        ...     ...
960         29      28
905         81      43
1096        17      13
235         13       7
1061        55      29

[1715 rows x 2 columns]
      medals
1794       0
104       51
693        0
1260       2
1489       0
...      ...
960        1
905        1
1096       0
235        0
1061       8

[1715 rows x 1 columns]


Now we will scale our x values

In [86]:
x_mean = x.mean()
x_std = x.std()
print(x_mean)
print(x_std)

athletes    73.201749
events      35.506706
dtype: float64
athletes    128.291352
events       49.613104
dtype: float64


In [87]:
x = (x - x_mean) / x_std
x.describe()

,athletes,events
count,1.715000e+03,1.715000e+03
mean,-4.557417e-17,4.971728e-17
std,1.000000e+00,1.000000e+00
min,-5.627951e-01,-6.955160e-01
25%,-5.160266e-01,-5.947361e-01
50%,-4.224895e-01,-4.536444e-01
75%,-3.275162e-02,1.711905e-01
max,5.969212e+00,4.726439e+00


In [88]:
x['intercept'] = 1
x = x[['intercept'] + predictors]
print(x)
print(x.T)

      intercept  athletes    events
1794          1 -0.555000 -0.675360
104           1  1.175436  1.400704
693           1 -0.555000 -0.675360
1260          1 -0.297773 -0.030369
1489          1 -0.531616 -0.614892
...         ...       ...       ...
960           1 -0.344542 -0.151305
905           1  0.060785  0.151035
1096          1 -0.438079 -0.453644
235           1 -0.469258 -0.574580
1061          1 -0.141878 -0.131149

[1715 rows x 3 columns]
              1794      104      693       1260      1489      712       950   \
intercept  1.00000  1.000000  1.00000  1.000000  1.000000  1.000000  1.000000   
athletes  -0.55500  1.175436 -0.55500 -0.297773 -0.531616  2.180960 -0.414695   
events    -0.67536  1.400704 -0.67536 -0.030369 -0.614892  2.267411 -0.614892   

               1733      1555      1691  ...      1278      1300      1202  \
intercept  1.000000  1.000000  1.000000  ...  1.000000  1.000000  1.000000   
athletes  -0.453669 -0.406900 -0.313363  ...  0.637598 -0.5316

In [89]:
lmbda = 2
I = np.identity(x.shape[1])

We don't want to penalize the y_intercept so we will do this 

In [90]:
I[0][0] = 0
penaltyMat = lmbda * I
penaltyMat

array([[0., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

We will use the ridge regression formula now 

In [91]:
w = np.linalg.inv(x.T@x + penaltyMat) @ x.T @ y
w.index = ['intercept','athletes','events']
print(w)

              medals
intercept  10.776093
athletes   62.435527
events    -34.741398


In [92]:
x_test = test[predictors]
x_test = (x_test - x_mean) / x_std
x_test['intercept'] = 1
x_test = x_test[['intercept'] + predictors]
x_test

,intercept,athletes,events
808,1,-0.484848,-0.493956
2000,1,-0.531616,-0.614892
1114,1,-0.422490,-0.393176
2036,1,-0.453669,-0.433488
1217,1,-0.461463,-0.514112
...,...,...,...
1535,1,-0.461463,-0.655204
82,1,-0.274389,0.009943
1468,1,-0.547206,-0.655204
1944,1,-0.165262,0.090567


I used the same mean and standard deviation I used in the training set because the data we will be using to make predictions is actually unknown 

In [93]:
predictions = x_test @ w
print(predictions)

         medals
808   -2.334887
2000  -1.053428
1114  -1.942761
2036  -2.488947
1217  -0.174632
...         ...
1535   4.727093
82    -6.700963
1468  -0.626275
1944  -2.688571
613   50.183304

[429 rows x 1 columns]


Comparing the model with scikit learn's Ridge Regression model

In [94]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=lmbda)
ridge.fit(x[predictors],y)
#we don't need to pass our intercept term in this scikit-learn's model

Ridge(alpha=2)

In [95]:
print(f'sklearns intercept and coefficients are {ridge.intercept_} and {ridge.coef_}')
print(f'Our models intecept and coefficients are {w}')

sklearns intercept and coefficients are [10.77609329] and [[ 62.43552721 -34.74139788]]
Our models intecept and coefficients are               medals
intercept  10.776093
athletes   62.435527
events    -34.741398


In [96]:
sklearn_predictions = ridge.predict(x_test[predictors])
predictions - sklearn_predictions

,medals
808,7.105427e-15
2000,-2.842171e-14
1114,2.309264e-14
2036,2.309264e-14
1217,-1.421085e-14
...,...
1535,-8.881784e-14
82,1.580958e-13
1468,-4.263256e-14
1944,1.376677e-13


We get very small values as the difference 

Now we just need to find the optimal lambda. For that we may need to wrap what we did in some functions 

In [123]:
def ridge_fit(train,predictors,target,lmbda):
    x = train[predictors].copy()
    y = train[target].copy()
    
    x_mean = x.mean()   
    x_std = x.std()

    x = (x - x_mean) / x_std
    x['intercept'] = 1
    x = x[['intercept']+predictors]
    penaltyMat = lmbda * np.identity(x.shape[1])
    penaltyMat[0][0] = 0

    w = np.linalg.inv(x.T@x + penaltyMat) @ x.T @ y
    w.index = ['intercept','athletes','events']
    return w, x_mean, x_std


In [124]:
def ridge_predict(test,predictors, x_mean,x_std,w):
    test_x = test[predictors]
    test_x = (test_x - x_mean) / x_std
    test_x['intercept'] = 1
    test_x = test_x[['intercept'] + predictors]

    predictions = test_x @ w
    return predictions

In [125]:
from sklearn.metrics import mean_absolute_error
#this will tell us how good our model is 
#this will tell us the difference between the actual and predicted y values
lmbdas = [10**i for i in range(-2,5)]
lmbdas

[0.01, 0.1, 1, 10, 100, 1000, 10000]

In [126]:
for i in lmbdas:
    w, x_mean, x_std = ridge_fit(train,predictors,target,i)
    predictions = ridge_predict(test,predictors,x_mean,x_std,w)
    print(mean_absolute_error(test[target],predictions))

6.271670503184058
6.268007576619037
6.23318596116771
6.063392242218722
7.2470256597695215
6.899447907723192
11.40797851498718


This helps us decide what the best lambda is. As you can as we increase our lambda, error goes up. First 3 or 4 values gives us the lambdas where we might be overfitting and last 2 or 3 values gives us the lambdas where we might be underfitting 

We will pick the lambda which is best, in essence gives us the least error